## Initialization

There are several ways to draw background maps with Python. For a complete review, visit the [map section](https://www.python-graph-gallery.com/map) of the gallery

This example uses the `Basemap` library. Let's initialize a map of the world as explained in [this post](https://www.python-graph-gallery.com/281-basic-map-with-basemap).

In [1]:
# libraries
from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt

# Set the plot size for this notebook:
plt.rcParams["figure.figsize"]=18,10

## 🔗  Great Circle

> A great circle is the intersection of the sphere and a plane that passes through the center point of the sphere. [Wikipedia](https://en.wikipedia.org/wiki/Great_circle)

Basically, a great circle shows the shortest path between 2 locations, knowing that our planet is a sphere. This path is not a straight line but an arc, which gives a much better appearance to the map.

Let's add a connection between London and New York. This is quite straightforward with `Basemap` thanks to the `drawgreatcircle()` function.

## Alliance Map Class

Create a map class that is a subclass of `Basemap`. Add some helper functions.

In [2]:
import pandas as pd

def is_float(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

class MyBasemap(Basemap):
    def __init__(self, *args, **kwargs):
        super().__init__()
        # load country attributes such as longitude, latitude, and BGN names
        self.countries_data = pd.read_csv("./version4.1_csv/cow.txt", sep=";", skiprows=28)
        self.countries_on_map = set()

    def printcountries(self, d: int=3, max_len: int=12, fontsize: int=5, min_size: float=1e5):
        """
        Label countries with their name on the map. Only label those with a land area 
        larger than `min_size` and print the first `max_len` char of the country names.
        """
        data = self.countries_data
        data = data[(data.latitude > self.llcrnrlat+d) & (data.latitude < self.urcrnrlat-d) & (data.longitude > self.llcrnrlon+d) & (data.longitude < self.urcrnrlon-d)]
        for ix, country in data.iterrows():
            if is_float(country.land_total) and float(country.land_total) >= min_size:
                self.countries_on_map.add(country.BGN_proper.lower())
                pos = self(country.longitude, country.latitude)
                plt.text(*pos, s=country.BGN_proper[:max_len], fontsize=fontsize)
                plt.plot([pos[0]], [pos[1]], marker="o", markersize=3, markeredgecolor="blue", markerfacecolor="blue")

    def get_country_row(self, country):
        """
        Query the attributes of `country`
        """
        ct = country.lower()
        if "united states of america".find(ct) >= 0:
            ct = "united states"
        found = False
        for country_name in self.countries_on_map:
            if country_name.find(ct) >= 0 or ct.find(country_name) >= 0:
                found = True
                break
        if not found:
            return None
        row = self.countries_data.loc[self.countries_data.BGN_name.apply(lambda x: x.lower().find(ct) >= 0 or ct.find(x.lower()) >= 0)]
        if len(row) > 0:
#             if len(row) > 1:
#                 print(ct, row.BGN_name.tolist())
            return row.iloc[0]

    def connect_countries(self, c0: str, c1: str):
        """
        Draw a great circle between the two countries `c0` and `c1` if 
        they are labelled on the map and their positions (latitude, longitude) can be found.
        """
        r0 = self.get_country_row(c0)
        r1 = self.get_country_row(c1)
        if r0 is None or r1 is None:
            return
        coord0 = self(r0.longitude.item(), r0.latitude.item())
        coord1 = self(r1.longitude.item(), r1.latitude.item())
        coord = coord0 + coord1
        self.drawgreatcircle(*coord, linewidth=1, color='#69b3a2')

In [3]:
# alliance = pd.read_csv("./version4.1_csv/alliance_v4.1_by_dyad_yearly.csv")
alliance = pd.read_csv("./version4.1_csv/alliance_v4.1_by_dyad.csv")

In [4]:
def get_year_map(m, year, key="dyad_st_year"):
    """
    Draw a great circle on the Basemap `m` between the 
    countries that established alliance in `year`.
    """
    year_data = alliance.loc[alliance[key] == year]
    for it, (idx, row) in enumerate(year_data.iterrows()):
#         print(row.state_name1, row.state_name2)
        m.connect_countries(row.state_name1, row.state_name2)

In [5]:
def plot_alliance(year):
    # Background map
    plt.figure()
    m=MyBasemap(llcrnrlon=-179, llcrnrlat=-60, urcrnrlon=179, urcrnrlat=70,  projection='merc')
    m.drawmapboundary(fill_color='white', linewidth=0)
    m.drawcoastlines(linewidth=0.1)
    m.drawcountries(linewidth=0.2)
    m.fillcontinents(color='#f2f2f2', alpha=0.7)
    m.drawcoastlines(linewidth=0.1, color="white")

    # Loop on every pair of cities to add the connection
    # for startIndex, startRow in df.iterrows():
    #     for endIndex in range(startIndex, len(df.index)):
    #         endRow = df.iloc[endIndex]
    #         m.drawgreatcircle(startRow.lon, startRow.lat, endRow.lon, endRow.lat, linewidth=1, color='#69b3a2')

    # Add city names
    # for i, row in df.iterrows():
    #     plt.annotate(row.city, xy=m(row.lon+3, row.lat), verticalalignment='center')

    m.printcountries(d=1, max_len=16)
    # m.connect_countries("United States", "Canada")
    # m.connect_countries('United States', 'Dominican Republic')
    get_year_map(m, year)
    plt.title(f"New alliances established in {year}")
    plt.tight_layout()
    plt.savefig(f"./img/{year}.png")
    plt.close('all')

In [6]:
# for year in range(1816, 2013):
#     plot_alliance(year)
plot_alliance(1962)